In [1]:
import pandas as pd
import csv
import numpy as np

df = pd.read_csv("Dataset(SMOTE).csv", low_memory=False)
df

,label,file_access,pe_imports,sig_clamav,pe_sec_character,sig_antivirus_virustotal,cmd_exec,api_resolv,pe_sec_entropy,pe_sec_name,...,sig_antianalysis_detectfile,sig_network_cnc_http,sig_removes_zoneid_ads,sig_downloader_cabby,sig_sniffer_winpcap,sig_creates_largekey,sig_infostealer_bitcoin,sig_static_detection,sig_antiav_servicestop,sig_recon_beacon
0,0,764,129,560,81,2280,102,1302,1147,211,...,2,12,5,2,2,3,0,231,4,11
1,0,984,129,560,81,2018,112,1302,1147,211,...,2,12,5,2,2,3,0,231,4,11
2,0,2164,1394,560,77,2018,541,237,124,207,...,2,12,5,2,2,3,0,231,4,11
3,0,946,962,560,81,2214,527,1212,183,211,...,2,12,5,2,2,3,0,231,4,11
4,0,1743,1345,560,77,216,144,634,73,207,...,2,12,5,2,2,3,0,231,4,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,4,666,561,560,106,852,541,634,2102,232,...,2,12,5,2,2,3,0,230,4,11
10096,4,1099,427,560,107,2018,541,634,309,233,...,2,12,5,2,2,3,0,231,4,11
10097,4,1856,1407,560,81,1573,541,639,1626,211,...,2,12,5,2,2,3,0,71,4,11
10098,4,1846,1373,560,102,331,541,634,1354,230,...,2,12,5,2,2,3,0,118,4,11


In [2]:
from sklearn.model_selection import train_test_split

# Separating the features from the target variable
X = df.iloc[:, 1:65]
y = df['label']

# Splitting the data into a train and a dev set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 0) 

In [3]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

rfc = RandomForestClassifier(max_depth=8, max_features='sqrt', n_estimators=200,
                             random_state=0)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=0)
boruta_selector.fit(np.array(X_train), np.array(y_train))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	64
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	9 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	10 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	11 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	12 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	13 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	14 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	15 / 100
Confirmed: 	36
Tentative: 	8
Rejected: 	20
Iteration: 	16 / 100
Confirmed: 	36
Tentative: 	8
Reject

BorutaPy(estimator=RandomForestClassifier(max_depth=8, n_estimators=108,
                                          random_state=RandomState(MT19937) at 0x17638AAD340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17638AAD340, verbose=2)

In [4]:
# Using the BorutaPy object to transform the features in the dataset.
X_important_train = boruta_selector.transform(np.array(X_train))
X_important_test = boruta_selector.transform(np.array(X_test)) 

In [5]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_boruta = xgb.XGBClassifier(random_state=0)

# Define search space
param_grid = { 'booster': ['gbtree', 'gblinear', 'dart'],
                'max_depth': [3, 4, 5],
                'gamma': [0, 0.25, 1],
                'reg_lamba': [0, 1.0, 10]
                'scale_pos_weight': [1, 3, 5],
                'learning_rate': [0.1, 0.01, 0.05]}

# Create GridSearchCV object
grid_xgb = GridSearchCV(xgb_boruta, param_grid, refit=True, n_jobs=-1, cv=10, verbose=3, scoring='accuracy')

# Fit the data
grid_xgb.fit(X_important_train, y_train)

# Print best parameters and accuracy score
print("Best parameters found: ", grid_xgb.best_params_)
print("Best accuracy score found: {:.4f}".format(grid_xgb.best_score_))

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
Best parameters found:  {'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best accuracy score found: 0.8896


In [8]:
# Instantiate model
xgb_model = xgb.XGBClassifier(booster='gbtree', learning_rate=0.1, max_depth=5, 
                             n_estimators=100, random_state=0)

# Train the model on training data
xgb_model.fit(X_important_train, y_train)

# Use the forest's predict method on the test data
xgb_predict = xgb_model.predict(X_important_test)

In [9]:
from sklearn import metrics

# Accuracy
print(metrics.accuracy_score(y_test, xgb_predict))
# Precision
print(metrics.precision_score(y_test, xgb_predict, average='macro'))
# Recall
print(metrics.recall_score(y_test, xgb_predict, average='macro'))
# F1 Score
print(metrics.f1_score(y_test, xgb_predict, average='macro'))

0.902970297029703
0.9121160358044615
0.9010105745323876
0.8982060157761855
